# Normal in vitro clock

The circadian clock of cyanobacteria *Synechococcus elongatus PCC 7942* can be reconstructed *in vitro*.
This protocol samples such an *in vitro* clock with uniform time intervals.

## Initialize the protocol

In [1]:
# current working dir is /var/lib/jupyter/notebooks
import opentrons.execute
from opentrons import protocol_api
from pipette_viscous import transfer_viscous,aspirate_viscous, dispense_viscous
import time, datetime, pytz    # datetime for logging and pytz for timezone conversion

# start the protocol context
protocol = opentrons.execute.get_protocol_api("2.11")

# home is required
protocol.home()

Out of bounds move: X=(418.00000000000006 motor controller, 416.9673140124148 deck) too high for limit 418.0
Out of bounds move: X=(418.00000000000006 motor controller, 416.9673140124148 deck) too high for limit 418.0


## Load labware

- The incubator is on slot 11
- The alluminum rack is mounted to the incubator
- The GEB tiprack is on slot 1
- The Bio-rad PCR plate is on slot 3
- The P10 1st gen pipette is on the left

In [2]:
incubator = protocol.load_module("temperature module", 11)

rack = incubator.load_labware("opentrons_24_aluminumblock_fisher_1.5ml_centrifuge")    # the rack is mounted upon the temp. module (don't specify slot)

tip_rack = protocol.load_labware("geb_taller_96_tiprack_10ul", '1')    # custom labware. see definition at labware/

plate = protocol.load_labware("biorad_96_wellplate_200ul_pcr", '3')

pipette = protocol.load_instrument("p10_single", "left", tip_racks = [tip_rack])

## Sampling and liquid settings

- The reaction tube is at A1 of the alluminum rack, with vol at least 6 * 19 = 114ul, round up to 150ul.
- The dye tube is at A3, with vol at least 2 * 19 = 38ul, round up to 100ul


In [3]:
SAMPLE_INT = 4 * 60    # time interval in minutes
SAMPLE_STEPS = 18    # plus the initial sampling, in total sample SAMPLE_STEPS + 1 times
SAMPLE_VOL = 6    # ul, volume per sample
SAMPLE_OFFSET = 8    # define the starting well
#OIL_VOL = 8    # ul    # now we know that adding oil is a bad idea. but the protocol behind it can still be useful

RXN_TUBE = rack.wells_by_name()["A1"]
DYE_TUBE = rack.wells_by_name()["A3"]
#OIL_TUBE = rack.wells_by_name()["A5"]

<div class="alert alert-warning">
    <h2>STOP!</h2>
    Before you proceed, please check:<br />
    Have you placed the required labware?<br />
    Have you put the right volume of liquid in required position?
</div>

## Turn on the incubator

In [4]:
incubator.set_temperature(30)    # in celcius

## Running

In [ ]:
def run(protocol:protocol_api.ProtocolContext):
    "Sampling the in vitro clock and seal with oil"
    
    protocol.set_rail_lights(True)    # to indicate start of protocols
    
    start_t = time.time()    # record starting time
    
    for i in range(SAMPLE_STEPS + 1) :
        # delay. except first sampling starts immediately
        if i > 0 :
            tot_wait_t = i * SAMPLE_INT * 60    # total waiting time from start in seconds
            delay_secs = start_t + tot_wait_t - time.time()    # how many more seconds to wait
            # infinite loop will block the process, which messes up with simulation, but actually ok here
            protocol.delay(seconds=delay_secs)
            
        # current well
        cur_well = plate.wells()[i + SAMPLE_OFFSET]
            
        # pipette the dye
        transfer_viscous(pipette, protocol, SAMPLE_VOL / 3, DYE_TUBE, cur_well)
        
        # pipette the sample
        transfer_viscous(pipette, protocol, SAMPLE_VOL, RXN_TUBE, cur_well, if_mix=True)
        
        # pipette oil
        #aspirate_viscous(pipette, protocol, OIL_VOL, OIL_TUBE, if_touch_tip=True)
        #dispense_viscous(pipette, protocol, OIL_VOL, cur_well, if_mix=False, disp_height=2.5)
        
        # log
        tz = pytz.timezone('America/Chicago')
        chicago_now = datetime.datetime.now(tz)
        print("Sampling # " + str(i + 1) + " finished on " + chicago_now.strftime("%m/%d/%y %H:%M:%S"))
    
    protocol.set_rail_lights(False)    # to indicate end of protocols
    
# actual running
run(protocol)

Out of bounds move: X=(418.00000000000006 motor controller, 416.9673140124148 deck) too high for limit 418.0


Sampling # 1 finished on 07/06/22 16:21:57
Sampling # 2 finished on 07/06/22 20:21:57
Sampling # 3 finished on 07/07/22 00:21:57
Sampling # 4 finished on 07/07/22 04:21:57
Sampling # 5 finished on 07/07/22 08:21:57
Sampling # 6 finished on 07/07/22 12:21:57
Sampling # 7 finished on 07/07/22 16:21:58


TODO:

- [x] Pipette oil
- [x] Dispense height too low now
- [x] Tune robot time. Robot time is read only. time zone cannot be set on a system-level